In [1]:
import csv
import cv2
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, MaxPooling2D, Dropout, Lambda
from keras.layers.convolutional import Conv2D
from keras.models import Model
from keras.layers import Cropping2D

Using TensorFlow backend.


In [2]:
lines = []
# Read the details from csv file
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    print(reader)
    for line in reader:
        lines.append(line)

In [3]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)
train_samples = np.array(train_samples)
validation_samples = np.array(validation_samples)

In [4]:
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)



In [5]:
    images= []
    measurements = []
    correction = 0.15
    for line in lines:
        for i in range(3):
            source_path = line[i]
            #print("Source_path",source_path)
            tokens = source_path.split('/')
            filename = tokens[-1]
            local_path = './data/IMG/'+filename
            image = cv2.imread(local_path)
            #cv2.imshow("image",image)
            #cv2.waitKey(0)
            images.append(image)
            measurement = float(line[3])
            if (i==0):
                measurements.append(measurement)
            elif (i==1):
                measurements.append(measurement+correction)
            elif(i==2):
                measurements.append(measurement-correction)

In [6]:
augumented_images = []
augumented_measurements = []

for image,measurement in zip(images,measurements):
    augumented_images.append(image)
    augumented_measurements.append(measurement)
    flipped_image = cv2.flip(image,1)
    flipped_measurement = float(measurement) * -1.0
    augumented_images.append(flipped_image)
    augumented_measurements.append(flipped_measurement)
    
X_train = np.array(augumented_images)
y_train = np.array(augumented_measurements)

In [7]:
# compile and train the model using the generator function
train_generator = generator(X_train, batch_size=32)
validation_generator = generator(y_train, batch_size=32)

In [8]:
ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/255.0 - 1.,
        input_shape=(160,320,3)))
#model = Sequential()
#model.add(Lambda(lambda x:x/255.0-0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(3,160,320)))
model.add(Conv2D(24,(5,5),strides=(2,2),activation='relu'))
model.add(Conv2D(36,(5,5),strides=(2,2),activation='relu'))
model.add(MaxPooling2D(strides=(1,1)))
model.add(Conv2D(48,(5,5),strides=(2,2),activation='relu'))
model.add(MaxPooling2D(strides=(1,1)))
model.add(Conv2D(64,(1,1),activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(1, 1),strides=(1,1)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1),strides=(1,1)))
model.add(Flatten())
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.summary()

model.compile(optimizer='adam',loss='mse')

model.fit(X_train,y_train, epochs = 10,validation_split=0.2,shuffle=True)

#model.fit_generator(train_generator, steps_per_epoch= len(train_samples), validation_data=validation_generator, validation_steps=len(validation_samples), epochs=3)

model.save('model.h5')

print('Model saved')

exit()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 77, 36)        21636     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 76, 36)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 36, 48)         43248     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 35, 48)         0         
__________